Read only stvari

In [1]:
import os
states=[
    'Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut',
    'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa',
    'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan',
    'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire',
    'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio',
    'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota',
    'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia',
    'Wisconsin', 'Wyoming'
]
main_folder=".."
trainset=os.path.join(main_folder,'d50States10k')
valset=os.path.join(main_folder,'d50States2K_test\\test_data')

Definisemo dataset

In [2]:
import torch
import torchvision
import numpy as np
import os 
import cv2
from torch.utils.data import Dataset,TensorDataset

def obradi(slika):
    r=slika[:,:,0]
    g=slika[:,:,1]
    b=slika[:,:,2]
    slika = np.stack([r, g, b], axis=0)
    slika=slika.astype(float)
    slika/=255
    slika=torch.from_numpy(slika)
    return slika

Klasa i inicijalizacija seta 

In [3]:
class LokacijeDataset(Dataset):
    def __init__(self,tip):
        imena=[] #Ime slike
        klase=[] #Broj klase
        for i in range(0,50):
            
            
            if tip=="train":
                put=os.path.join(trainset,states[i])
            else:
                put=os.path.join(valset,states[i])
            slike=os.listdir(put)
            N=len(slike)
            for j in range(0,N,4):
               # print(j,tip)
                #print(states[i],slike[j],slike[j][:-7])
                imena.append(slike[j][:-6])
                klase.append(i)
        self.klase=klase
        self.imena=imena
    def __len__(self):
        return len(self.klase)
    def __getitem__(self,idx):
        ime = self.imena[idx]
        klasa = self.klase[idx]
        #print(klasa,ime)
        ret=[]
        for i in range(0,360,90):
            #print(ime)
            trenime=ime+"_"+str(i)+".jpg"
            put=os.path.join(trainset,states[klasa])
            if not os.path.exists(os.path.join(put,trenime)):
                put=os.path.join(valset,states[klasa])
            put=os.path.join(put,trenime)
           # print(put)
            slika=cv2.imread(put)
            slika=obradi(slika)
            ret.append(slika)
        del slika
        return { 
            'data':torch.stack(ret),
            'label':self.klase[idx]
        }

train_set = LokacijeDataset("train")
train_set,val_set=torch.utils.data.random_split(train_set,[0.9,0.1])
test_set= LokacijeDataset("test")

In [4]:
print(len(train_set))
print(len(val_set))
print(len(test_set))

112500
12499
24998


In [5]:
if torch.cuda.is_available():
    device="cuda"
else:
    device="cpu"
    print("NE RADI CUDA!")

Inicijalizacija mreze

In [6]:
from copy import deepcopy
import torch.nn as nn
from torchvision import models
import torch.nn as nn
import torch

vgg = models.vgg16_bn(pretrained=True)
for param in vgg.parameters():
    param.requires_grad = False

class Mreza (nn.Module):
    def __init__ (self):
        super().__init__()
        self.features = deepcopy(vgg.features)
        self.avgpool = deepcopy(vgg.avgpool)
        self.konvolucija=nn.Sequential(
            self.features,
            self.avgpool,
            nn.Flatten()
        )

        self.povezan=nn.Sequential(
            nn.Linear (100352, 1000),
            nn.BatchNorm1d(1000),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1000, 450),
            nn.BatchNorm1d(450),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(450, 50),
        )
        self.povezan.to(device)
        self.konvolucija.to(device)

    def forward (self, x):
       # print(x.size())
        x=x.type(torch.float)  
        a=self.konvolucija(x[:,0,:,:,:])
        b=self.konvolucija(x[:,1,:,:,:])
        c=self.konvolucija(x[:,2,:,:,:])
        d=self.konvolucija(x[:,3,:,:,:])
       # print(a,b,c,d)
        tens=torch.cat([a,b,c,d],1)
        del a,b,c,d
        #print(tens)
        return self.povezan(tens)
model = Mreza()

c:\Users\psiml\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\psiml\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_BN_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_BN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Inicijalizacija parametara

In [7]:
broj_epoha = 50
batch_velicina=100
loss_funk = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
loader = torch.utils.data.DataLoader(dataset=train_set,batch_size=batch_velicina,shuffle=True,num_workers=0,drop_last=True,pin_memory=True)
val_loader = torch.utils.data.DataLoader(dataset=val_set,batch_size=batch_velicina,shuffle=False,num_workers=0,drop_last=False,pin_memory=True)

Testiranje

In [8]:
def totensor(y):
    ret=np.zeros((len(y),50))
    for i in range(0,len(y)):
        ret[i][round(y[i].item())]=1
    ret= torch.Tensor(ret)
    ret=ret.to(device)
    return ret

def test(model,sample):
    #print(sample)
    sample=sample.to(device)
    rez=model(sample.reshape(1,4,3,256,256))
    #print(rez)
    _,b=torch.max(rez,dim=1)
    #print(rez,b)
    return b

def get_accuracy(y_hat,y):
    y_hat = y_hat.cpu().detach().numpy()
    y=y.cpu().numpy()
    #print(y_hat)
    y_hat=np.argmax(y_hat,axis=1)
    #print(y_hat)
    corrects=(y_hat==y).sum()
    #print(y_hat,y,corrects)
    return corrects.item()

def validation(model):
    with torch.no_grad():
        model.eval()
        kol=0
        total_loss=0
        for batch_num, lokacije in enumerate(val_loader):
            x=lokacije['data']
            y=lokacije['label']
            x = x.to(device)
            y = y.to(device)
                #print(y)
            y_hat = model(x)
            loss=loss_funk(y_hat,totensor(y))
            total_loss+=loss.detach().item()
           # print(np.argmax(y_hat.cpu().detach().numpy(),axis=1),y)
            kol+=get_accuracy(y_hat=y_hat, y=y)
            #print(kol)
            del y_hat
        kol=kol*100/len(val_set)
        total_loss/=(batch_num+1)
    return total_loss,kol

def test_weights(net):
    print(list(net.parameters())[-6])

def test_out():
    with torch.no_grad():
        a = val_set[100]['data'].reshape(1,4,3,256,256).to(device)
        tmp = model(a)
    print(tmp)

Treniranje

In [9]:
best_model = model
best_acc=0

import copy
import time

import datetime
ime="TrainingLogs\\VGG_TRAINING_"
ime+=datetime.datetime.now().strftime("%d_%m_%Y_%H_%M_%S")
ime+=".txt"
#print(ime)
ouf = open(ime,"w")  
for epoha in range(1,broj_epoha+1):
    train_running_loss =0.0
    train_acc=0.0
    model = model.train()
    
    start = time.time()
    for batch_num, lokacije in enumerate(loader):
            #test_out()
           # print("RADI")
            x=lokacije['data']
            y=lokacije['label']
            x = x.to(device)
            y = y.to(device)
           # print(x)
            optimizer.zero_grad()
            y_hat = model(x)
            #print(y_hat.get_device())
            #print(y.get_device())
           # print(y_hat,totensor(y))
            loss = loss_funk(y_hat, totensor(y))
            
            loss.backward()
            optimizer.step()


            train_running_loss += loss.detach().item()
            train_acc += get_accuracy(y_hat=y_hat, y=y)
            del y_hat
            del x
            del y 
            del loss

            print(f'Epoch: {epoha}\t{100 * (batch_num + 1) / len(loader):.2f}% complete. {time.time() - start:.2f} seconds elapsed in epoch. Time until completion {(time.time()-start)/((batch_num + 1) / len(loader)) - time.time()+start:.2f} seconds.',
                end='\r')

    print(f'',end='\n')    
    epoch_loss = train_running_loss / (batch_num+1)
    epoch_acc = 100*train_acc / ((batch_num+1)*batch_velicina)
    val_loss,val_acc = validation(model)
    if val_acc>best_acc:
        best_acc=val_acc
        best_model=copy.deepcopy(model)
        torch.save(best_model,'VGGBestModel.pt')
        ouf.write("NEW BEST MODEL: ")
        print("NEW BEST MODEL: ",end='')

    ouf.write('Epoch: %d | Train Loss: %.4f | Train Accuracy: %.2f | Validation Loss: %.4f | Validation Accuracy: %.2f' \
          %(epoha, epoch_loss, epoch_acc,val_loss,val_acc))
    ouf.write("\n")
    print('Epoch: %d | Train Loss: %.4f | Train Accuracy: %.2f | Validation Loss: %.4f | Validation Accuracy: %.2f' \
          %(epoha, epoch_loss, epoch_acc,val_loss,val_acc))

Epoch: 1	100.00% complete. 3211.04 seconds elapsed in epoch. Time until completion 0.00 seconds...
NEW BEST MODEL: Epoch: 1 | Train Loss: 2.8684 | Train Accuracy: 21.26 | Validation Loss: 2.4053 | Validation Accuracy: 32.26
Epoch: 2	100.00% complete. 2413.75 seconds elapsed in epoch. Time until completion 0.00 seconds...
NEW BEST MODEL: Epoch: 2 | Train Loss: 2.2351 | Train Accuracy: 35.82 | Validation Loss: 2.2493 | Validation Accuracy: 36.52
Epoch: 3	100.00% complete. 2493.48 seconds elapsed in epoch. Time until completion 0.00 seconds...
NEW BEST MODEL: Epoch: 3 | Train Loss: 1.8175 | Train Accuracy: 46.60 | Validation Loss: 2.2384 | Validation Accuracy: 37.14
Epoch: 4	100.00% complete. 2453.31 seconds elapsed in epoch. Time until completion 0.00 seconds...
NEW BEST MODEL: Epoch: 4 | Train Loss: 1.4418 | Train Accuracy: 56.39 | Validation Loss: 2.2778 | Validation Accuracy: 37.42
Epoch: 5	100.00% complete. 2528.42 seconds elapsed in epoch. Time until completion 0.00 seconds...
Epoch

KeyboardInterrupt: 

In [ ]:
for i in range(0,len(val_set)):
    a=val_set[i]['data']
    if(test(model,a)==val_set[i]['label']):
        print(i)

In [ ]:
def validation(model):
    with torch.no_grad():
        model.eval()
        kol=0
        total_loss=0
        for batch_num, lokacije in enumerate(val_loader):
            x=lokacije['data']
            y=lokacije['label']
            x = x.to(device)
            y = y.to(device)
                #print(y)
            y_hat = model(x)
            loss=loss_funk(y_hat,totensor(y))
            total_loss+=loss.detach().item()
           # print(np.argmax(y_hat.cpu().detach().numpy(),axis=1),y)
            kol+=get_accuracy(y_hat=y_hat, y=y)/len(y)
            #print(kol)
            del y_hat
        print(batch_num)
        kol=kol*100/(batch_num+1)
        #total_loss/=(batch_num+1)
    return total_loss,kol

validation(model)

In [ ]:
val_loader = torch.utils.data.DataLoader(dataset=val_set,batch_size=500,shuffle=False,num_workers=0,drop_last=False,pin_memory=True)

def get_accuracy(y_hat,y):
    y_hat = y_hat.cpu().detach().numpy()
    y=y.cpu().numpy()
    #print(y_hat)
    y_hat=np.argmax(y_hat,axis=1)
    #print(y_hat)
    corrects=(y_hat==y).sum()
    #print(y_hat,y,corrects)
    return corrects.item()

def vall(model):
    with torch.no_grad():
        model.eval()
        kol=0
        for batch_num, lokacije in enumerate(val_loader):
            x=lokacije['data']
            y=lokacije['label']
            x = x.to(device)
            y = y.to(device)
                #print(y)
            #print(x.size())
            y_hat = model(x)
            #print(y_hat)
            #print(y_hat,y)
            kol+=get_accuracy(y_hat=y_hat, y=y)
            del y_hat
    return kol

vall(m2)

In [10]:
print(validation(best_model))

(2.277770133972168, 37.41899351948156)


In [ ]:
torch.save(best_model,'VGGBestModel.pt')

In [ ]:
m2 = torch.load('VGGBestModel.pt')

In [ ]:
validation(m2)

In [11]:
test_loader = torch.utils.data.DataLoader(dataset=test_set,batch_size=batch_velicina,shuffle=False,num_workers=0,drop_last=False,pin_memory=True)


gmatrix = []
labele=[]
for batch_num, lokacije in enumerate(test_loader):
    x=lokacije['data']
    y=lokacije['label']
    x = x.to(device)
    for i in range(1,len(y)):
        labele.append(y[i].cpu().detach().numpy())
    y_hat = best_model(x)
    y_hat= y_hat.cpu().detach().numpy()
    y_hat=np.argmax(y_hat,axis=1)
    for i in range(0,len(y_hat)):
       gmatrix.append(y_hat[i])
    #print(gmatrix)
gmatrix=np.array(gmatrix)
labele=np.array(labele)
print(gmatrix,labele)

[ 9  8 23 ...  2 42 43] [ 0  0  0 ... 49 49 49]


In [12]:
print(len(labele),len(gmatrix),len(test_set))
for i in range(len(labele),len(gmatrix)):
    labele=np.append(labele,49)
print(len(labele),len(gmatrix),len(test_set))
kol=0
for i in range(1,len(gmatrix)):
    if gmatrix[i]==labele[i]:
        kol+=1
print(kol)

24748 24998 24998
24998 24998 24998
7572


In [19]:
ouf = open("VGGRezultati.txt","w")
for i in range(1,len(gmatrix)):
    ouf.write(str(round(gmatrix[i]))+" "+str(labele[i])+"\n")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
cf_matrix = confusion_matrix(labele, gmatrix)

for i in range(0, 50):
    p = np.sum(cf_matrix[i])
    cf_matrix[i] = cf_matrix[i] * 100 / p

plt.figure(figsize=(80, 45))
sns.heatmap(cf_matrix/100, annot=True, fmt='.2%', cmap='Blues')
plt.show()